In [21]:
import numpy as np
% tensorflow_version 2.x
import tensorflow as tf
import matplotlib.pyplot as plt
import time

In [ ]:
(train_images, train_labels),(test_images, test_labels) = tf.keras.datasets.cifar10.load_data()


170500096/170498071 [==============================] - 4s 0us/step


In [ ]:
print("The shape of the image set: ", train_images.shape)
print("The shape of the label set: ", train_labels.shape)
print("Print a label: ", train_labels[1])
name_dict = {0: "airplane",1:"automobile", 2: "bird", 3: "cat", 4: "deer", 5: "dog", 6: "frog", 7: "horse", 8: "ship", 9: "truck"}
fig, ax = plt.subplots(1,5)

for count in range(5):
  img = train_images[count]
  label = int(train_labels[count])
  ax[count].imshow(img)
  ax[count].set_title(name_dict[label])
  ax[count].axis("off")

## Preprocessing

Perform necessary preprocessing steps to prepare the images and labels.

In [ ]:
train_images = tf.data.Dataset.from_tensor_slices(train_images)

#normalizing image
train_images = train_images.map(lambda img: img/255)

train_labels = tf.data.Dataset.from_tensor_slices(train_labels.reshape((-1,)))
train_labels = train_labels.map(lambda label: tf.one_hot(label,10))

training_data = tf.data.Dataset.zip((train_images, train_labels))
training_data = training_data.batch(64)
training_data = training_data.shuffle(buffer_size = 64)
training_data = training_data.prefetch(16)

test_images = tf.data.Dataset.from_tensor_slices(test_images)

#normalizing image
test_images = test_images.map(lambda img: img/255)

test_labels = tf.data.Dataset.from_tensor_slices(test_labels.reshape((-1,)))
test_labels = test_labels.map(lambda label: tf.one_hot(label,10))

test_data = tf.data.Dataset.zip((test_images, test_labels))
test_data = test_data.batch(64)
test_data = test_data.shuffle(buffer_size = 64)
test_data = test_data.prefetch(16)

# Model
## Model 1 ResNet

In [ ]:
class ResidualBlock(tf.keras.layers.Layer):
  def __init__(self):
    super(ResidualBlock,self).__init__()

    #Define the different layers we need
    self.block = [tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, 
                                        kernel_regularizer = tf.keras.regularizers.l2(0.01), padding = 'same'),
                  tf.keras.layers.BatchNormalization(),
                  tf.keras.layers.activations.relu,

                  tf.keras.layers.Conv2D(filters = 32, kernel_size = 3,
                                        kernel_regularizer = tf.keras.regularizers.l2(0.01), padding = 'same'),
                  tf.keras.layers.BatchNormalization(),
                  tf.keras.layers.activations.relu
    ]

  def call(self, input_x):
    x = input_x
    for layer in self.block:
      x = layer(x)

    return x

In [ ]:
class ResNet(tf.keras.Model):
  def __init__(self,numb_of_blocks):
    super(ResNet, self).__init__()

    self.blocks = []
    self.blocks.append(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, activation = tf.keras.activations.relu,
                                        kernel_regularizer = tf.keras.regularizers.l2(0.01), padding = 'same'))
    for _ in range(numb_of_blocks):
      self.blocks.append(ResidualBlock())
    
    self.post_blocks = [
                        tf.keras.layers.GlobalAveragePool(),
                        tf.keras.layers.Dense(units = 10, activation= tf.keras.acitvations.softmax)
    ]

  def call(self, input_x):
    x = input_x

    for b in self.blocks:
      x = b(x) + x
    
    for layer in self.post_blocks:
      x = layer(x)

    return x

## Model 2 DenseNet:

In [ ]:
class TransitionLayers(tf.keras.layers.Layer):
  def __init__(self):
    super(TransitionLayers,self).__init__()

    #which layers should I use?
    self.layers = [
                   tf.keras.layers.Conv2d(filters = 64, kernel_size = 1),
                   tf.keras.layers.MaxPool2D()
    ]
  
  def call(self, input_x):
    x = input_x

    for layer in self.layers:
      x = layer(x)
    
    return x

#class that describes one element of the block
class Block(tf.keras.layers.Layer):
  def __init__(self):
    super(Block,self).__init__()

    self.block = [
             tf.keras.layers.Conv2d(filters = 32),
             tf.keras.layers.BatchNormalization(),
             tf.keras.layers.activations.relu
    ]
    
    self.concatenate = tf.keras.layers.Concatenate()

  def call(self, input_x):
    x = input_x

    for layer in self.block:
      x = self.concatenate([layer(x),x])
    
    return x

# class represents one denseblock consisting of multiple blocks
class DenseBlock(tf.keras.layers.Layer):
  def __init__(self, nr_blocks):
    super(DenseBlock,self).__init__()

    self.block = [Block() for _ in range(nr_blocks)]


  def call(self, input_x):
    x = input_x

    for block in self.block:
      x = block(x)

    return x

#the whole DenseNEt
class DenseNet(tf.keras.Model):
  def __init__(self, nr_blocks):
    super(DenseNet,self).__init__()

    self.blocks = []
    self.blocks.append(tf.keras.layers.Conv2d(filters=125, activation = tf.keras.activations.relu, kernel_size = 3, padding = "same",
                                              kernel_regularizer = tf.keras.regularizers.l2(0.01))

    for i in range(nr_blocks):
      self.blocks.append(DenseBlock())
      
      if (i < nr_blocks-1):
        self.blocks.append(TransitionLayers())

    self.blocks.append(tf.keras.layers.Flatten())
    self.blocks.append(tf.keras.layers.Dense(units = 10, activation = tf.keras.activations.softmax))

  def call(self, input_x):
    x = input_x

    for layer in self.block:
      x = layer(x) 

    return x

#Training


In [ ]:
def train_step(model, input, target, loss_function, optimizer, training=True):
  with tf.GradientTape() as tape:
    prediction = model(input, training)
    loss = loss_function(prediction,target) + tf.reduce_sum(model_losses)
    accuracy = (np.argmax(axis =1, target) == np.argmax(axis=1, prediction)) / target.shape[0]
    gradients = tape.gradient(loss,model.trainable_variables)
  
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  return loss, accuracy


def test(model, input_data, loss_function, training= False):
  losses = []
  accuracies = []

  for (image,label) in input_data:
    predicition = model(image, training)
    
    #calculate the loss
    loss = loss_function(prediction,label)

    #calculate the accuracy by comparing the predicted label (of the model) to the ground truth (label)
    accuracy = (np.argmax(prediction, axis=1) == np.argmax(target,axis=1))

    losses.append(loss.numpy())
    accuracies.appen(mean(accuracy))
  
  loss = np.mean(losses)
  accuracy = np.mean(accuracies)
  return loss, accuracy


In [ ]:
tf.keras.backend.clear_session()

resnet = ResNet(numb_of_blocks = 3)
densenet = DenseNet(nr_blocks = 3)

epochs = 30
learning_rate = 0.001
loss_function = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate)

running_Average_factor = 0.95

steps = []
#lists for both models
resnet_losses = []
resnet_accuracies = []

densenet_losses = []
densenet_accuracies = []

#lists for testing both models
res_test_losses = []
res_test_accuracies = []

dense_test_losses = []
dense_test_accuracies = []


#pretraining on training_data
pre_loss, preaccur = test(resnet, training_data, loss_function)
resnet_losses.append(pre_loss)
resnet_accuracies.append(preaccur)

pre_loss, preaccur = test(densenet, training_data, loss_function)
densenet_losses.append(pre_loss)
densenet_accuracies.append(preaccur)

#pretraining test on test data:
pre_loss, preaccur = test(resnet, test_data, loss_function)
res_test_losses.append(pre_loss)
res_test_accuracies.append(preaccur)

pre_loss, preaccur = test(densenet, test_data, loss_function)
dense_test_losses.append(pre_loss)
dense_test_accuracies.append(preaccur)


#actually start training
for epoch in epochs:
  steps.append(epoch)

  train_dataset = train_dataset.shuffle(buffer_size=64)
  test_dataset = test_dataset.shuffle(buffer_size=64)

  running_accuracy_res = 0
  running_accuracy_dense = 0
  running_loss_res = 0
  running_accuracy_dense = 0

  # train the model on each input image
  #compute the current loss and accuracy for both models
  for(input,label) in train_dataset:
    res_loss,res_accur = train(resnet, input, label, loss_function, optimizer)
    dense_loss,dense_accur = train(densenet, input, label, loss_function, optimizer)

    running_accuracy_dense = running_Average_factor * running_accuracy_dense + (1 - running_Average_factor) * dense_accur
    running_accuracy_res = running_Average_factor * running_accuracy_res + (1 - running_Average_factor) * res_accur

    running_loss_res = running_Average_factor * running_loss_res + (1 - running_Average_factor) * res_loss
    running_loss_dense = running_Average_factor * running_loss_dense + (1 - running_Average_factor) * dense_loss
  
  # save the computed values (for the epoch)
  resnet_accuracies.append(running_accuracy_res)
  densenet_accuracies.append(running_accuracy_dense)

  resnet_losses.append(running_loss_res)
  densenet_losses.append(running_loss_dense)

  #check how well the models do on the testing data
  test_loss, test_accuracy = test(resnet, test_dataset, loss_function, False)
  res_test_losses.append(test_loss)
  res_accuracies.append(test_accuracy)

  test_loss_d, test_accuracy_d = test(densenet, test_dataset, loss_function, False)
  dense_test_losses.append(test_loss_d)
  dense_test_accuracies.append(test_accuracy_d)


  #print current performance data
  print('RESNET: training_loss: ', running_loss_res.numpy(), 'training_accuracy: ', running_accuracy_res)
  print('DENSENET: training_loss: ', running_loss_dense.numpy(), 'training_accuracy: ', running_accuracy_dense))
  print('RESNET: test_loss: ', test_loss, 'test_accuracy: ', test_accuracy)
  print('RESNET: test_loss: ', test_loss_d, 'test_accuracy: ', test_accuracy_d)

In [ ]:

#do the visualization
#test loss and training loss
plt.figure()
line1, = plt.plot(resnet_losses)
line2, = plt.plot(densenet_losses)
line3, = plt.plot(res_test_losses)
line4, = plt.plot(dense_test_losses)

plt.xlabel("Training steps")
plt.ylabel("Loss")
plt.legend((line1,line2, line3, line4),("ResNet training", "DenseNEt training", "ResNet test", "DenseNet test"))
plt.show()

In [ ]:
#do the visualization
#test loss and training loss
plt.figure()
line1, = plt.plot(resnet_accuracies)
line2, = plt.plot(densenet_accuracies)
line3, = plt.plot(res_test_accuracies)
line4, = plt.plot(dense_test_accuracies)

plt.xlabel("Training steps")
plt.ylabel("Accuracy")
plt.legend((line1,line2, line3, line4),("ResNet training", "DenseNEt training", "ResNet test", "DenseNet test"))
plt.show()